In [2]:
import os

from dotenv import load_dotenv
from sqlmodel import create_engine

load_dotenv()
engine = create_engine(os.environ.get("POSTGRES_URL"))

In [1]:
import uuid
import datetime
import enum
from typing import Optional

from sqlmodel import Field, SQLModel


class VehicleModel(SQLModel, table=True):
    __tablename__ = "vehicle_model"
    __table_args__ = {'extend_existing': True}
    id: Optional[uuid.UUID] = Field(default=None, primary_key=True)
    capacity: int
    created_at: Optional[datetime.datetime] = Field(default_factory=datetime.datetime.utcnow, nullable=False)
    vehicle_class: int
    make: str
    model: str

    class VehicleType(enum.Enum):
        CAR = "CAR"
        MOTORCYCLE = "MOTORCYCLE"

    type: VehicleType

    class VehiclePropulsion(enum.Enum):
        PETROL = "PETROL"
        HYBRID = "HYBRID"
        ELECTRIC = "ELECTRIC"
        DIESEL = "DIESEL"

    propulsion: VehiclePropulsion

In [24]:
from sqlmodel import Session

sigra = VehicleModel(
    make="Daihatsu",
    model="Sigra",
    propulsion=VehicleModel.VehiclePropulsion.PETROL,
    type=VehicleModel.VehicleType.CAR,
    vehicle_class=1,
    capacity=7,
)

with Session(engine) as session:
    session.add(sigra)
    session.commit()

OperationalError: (psycopg2.OperationalError) could not receive data from server: Connection timed out
SSL SYSCALL error: Connection timed out

[SQL: INSERT INTO vehicle_model (capacity, created_at, vehicle_class, make, model, type, propulsion) VALUES (%(capacity)s, %(created_at)s, %(vehicle_class)s, %(make)s, %(model)s, %(type)s, %(propulsion)s) RETURNING vehicle_model.id]
[parameters: {'capacity': 7, 'created_at': datetime.datetime(2025, 1, 5, 9, 26, 29, 307851), 'vehicle_class': 1, 'make': 'Daihatsu', 'model': 'Sigra', 'type': 'CAR', 'propulsion': 'PETROL'}]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [4]:
from typing import List
from sqlmodel import select, or_
from sqlmodel import Session


def get_all_vehicle_models():
    statement = select(VehicleModel).where(or_(VehicleModel.make == "Daihatsu", VehicleModel.make == "Toyota"))
    results: List[VehicleModel] = session.execute(statement).scalars().all()
    return results


with Session(engine) as session:
    results = get_all_vehicle_models()
    print(results)
    # for i in results:
    #     print(type(i))
    #     print(i.capacity)
    #     i.capacity = i.capacity - 1
    # session.commit()
    # print(get_all_vehicle_models())

[VehicleModel(capacity=5, id=UUID('b6c0571f-20cf-43df-8263-24f9b0a43b12'), vehicle_class=0, model='Calya', propulsion=<VehiclePropulsion.PETROL: 'PETROL'>, created_at=datetime.datetime(2024, 10, 12, 16, 31, 44, 292602, tzinfo=datetime.timezone.utc), make='Toyota', type=<VehicleType.CAR: 'CAR'>), VehicleModel(capacity=3, id=UUID('dab5c3a3-1e59-4a35-a9ee-6925406f1d07'), vehicle_class=0, model='Ayla', propulsion=<VehiclePropulsion.PETROL: 'PETROL'>, created_at=datetime.datetime(2024, 11, 9, 17, 30, 20, 506742, tzinfo=datetime.timezone.utc), make='Daihatsu', type=<VehicleType.CAR: 'CAR'>)]


In [23]:
import time

with Session(engine) as session:
    statement = select(VehicleModel).where(VehicleModel.model == "Sigra")
    results = session.exec(statement)
    car = results.one()
    print("Car: ", car)
    time.sleep(1)
    input("Confirm delete?")
    session.delete(car)
    session.commit()
    print(get_all_vehicle_models())

Car:  vehicle_class=1 id=UUID('7e69ea7d-dc9f-4aa4-8856-8c2ec3bd632c') capacity=5 model='Sigra' propulsion=<VehiclePropulsion.PETROL: 'PETROL'> created_at=datetime.datetime(2025, 1, 5, 7, 35, 6, 132653, tzinfo=datetime.timezone.utc) make='Daihatsu' type=<VehicleType.CAR: 'CAR'>
[VehicleModel(vehicle_class=0, id=UUID('b6c0571f-20cf-43df-8263-24f9b0a43b12'), capacity=5, model='Calya', propulsion=<VehiclePropulsion.PETROL: 'PETROL'>, created_at=datetime.datetime(2024, 10, 12, 16, 31, 44, 292602, tzinfo=datetime.timezone.utc), make='Toyota', type=<VehicleType.CAR: 'CAR'>), VehicleModel(vehicle_class=0, id=UUID('dab5c3a3-1e59-4a35-a9ee-6925406f1d07'), capacity=3, model='Ayla', propulsion=<VehiclePropulsion.PETROL: 'PETROL'>, created_at=datetime.datetime(2024, 11, 9, 17, 30, 20, 506742, tzinfo=datetime.timezone.utc), make='Daihatsu', type=<VehicleType.CAR: 'CAR'>)]


In [2]:
import uuid
from typing import Optional

from sqlmodel import Field, SQLModel, Session, select, Relationship


class VehicleModel(SQLModel, table=True):
    __tablename__ = "vehicle_model"
    __table_args__ = {'extend_existing': True}

    id: Optional[uuid.UUID] = Field(default=None, primary_key=True)
    # capacity: int
    # created_at: Optional[datetime.datetime] = Field(default_factory=datetime.datetime.utcnow, nullable=False)
    # vehicle_class: int
    # make: str
    # model: str
    #
    # class VehicleType(enum.Enum):
    #     CAR = "CAR"
    #     MOTORCYCLE = "MOTORCYCLE"
    # type: VehicleType
    #
    # class VehiclePropulsion(enum.Enum):
    #     PETROL = "PETROL"
    #     HYBRID = "HYBRID"
    #     ELECTRIC = "ELECTRIC"
    #     DIESEL = "DIESEL"
    # propulsion: VehiclePropulsion
    vehicle_units: list["VehicleUnit"] = Relationship(back_populates="vehicle_model")


class VehicleUnit(SQLModel, table=True):
    __tablename__ = "vehicle_unit"
    __table_args__ = {'extend_existing': True}

    id: Optional[uuid.UUID] = Field(default=None, primary_key=True)
    # driver_id: uuid.UUID = Field(foreign_key="driver.id")
    # vehicle_year: int
    # vehicle_color: str
    # license_plate: str
    # deleted_at: datetime.datetime | None = Field(default=None)
    # updated_at: datetime.datetime | None = Field(
    #     default_factory=lambda: datetime.datetime.now(datetime.timezone.utc),
    #     nullable=False,
    #     sa_column_kwargs={"onupdate": lambda: datetime.datetime.now(datetime.timezone.utc)},
    # )
    # created_at: Optional[datetime.datetime]
    # photo_1: str | None = Field(default=None)
    # photo_2: str | None = Field(default=None)
    # photo_3: str | None = Field(default=None)
    vehicle_id: uuid.UUID = Field(default=None, foreign_key="vehicle_model.id")
    vehicle_model: "VehicleModel" = Relationship(back_populates="vehicle_units")


def get_all_vehicle_units():
    statement = select(VehicleUnit)
    results: List[VehicleUnit] = session.execute(statement).scalars().all()
    return results


with Session(engine) as session:
    results = get_all_vehicle_units()
    print(results)
    print([i.vehicle_model.model for i in results])



[VehicleUnit(vehicle_id=UUID('dab5c3a3-1e59-4a35-a9ee-6925406f1d07'), id=UUID('6da38e1b-d9e8-4e68-9c50-6ebca6fa9ae4'))]


AttributeError: 'VehicleModel' object has no attribute 'model'

In [42]:
 with Session(engine) as session:
    statement = select(VehicleUnit, VehicleModel).join(VehicleModel)
    results = session.exec(statement).all()
    print(results)
    for vu, vm in results:
        print("Vehicle Model:", vm, "\nVehicle Unit:", vu, "\n")

[(VehicleUnit(vehicle_color='Silver', driver_id=UUID('ca65506a-6ce8-492d-9c64-e0047bdb7723'), deleted_at=None, updated_at=datetime.datetime(2025, 1, 5,  ... (183 characters truncated) ... -1e59-4a35-a9ee-6925406f1d07'), created_at=datetime.datetime(2025, 1, 5, 9, 27, 21, 957164, tzinfo=datetime.timezone.utc), photo_1=None, photo_3=None), VehicleModel(id=UUID('dab5c3a3-1e59-4a35-a9ee-6925406f1d07'), vehicle_class=0, model='Ayla', propulsion=<VehiclePropulsion.PETROL: 'PETROL'>, make='Daihatsu', created_at=datetime.datetime(2024, 11, 9, 17, 30, 20, 506742, tzinfo=datetime.timezone.utc), capacity=3, type=<VehicleType.CAR: 'CAR'>))]
Vehicle Model: vehicle_color='Silver' driver_id=UUID('ca65506a-6ce8-492d-9c64-e0047bdb7723') deleted_at=None updated_at=datetime.datetime(2025, 1, 5, 9, 27, 21, 957164, tzinfo=datetime.timezone.utc) photo_2=None id=UUID('6da38e1b-d9e8-4e68-9c50-6ebca6fa9ae4') vehicle_year=2020 license_plate='B1234ABC' vehicle_id=UUID('dab5c3a3-1e59-4a35-a9ee-6925406f1d07') crea

In [44]:
 with Session(engine) as session:
    statement = select(VehicleUnit)
    results: List[VehicleUnit] = session.exec(statement).scalars().all()
    print(results)
    for vu in results:
        print(vu.vehicle_model.model, "\n")

InvalidRequestError: One or more mappers failed to initialize - can't proceed with initialization of other mappers. Triggering mapper: 'mapped class VehicleUnit->vehicle_unit'. Original exception was: Mapper 'mapped class VehicleModel->vehicle_model' has no property 'vehicle_model'